# Extração e limpeza dos dados da LOA

**objetivo**: identificar as emendas propostas pela legislatura 2021-2024 e cruzar com o vereador/partido que propôs.

Scripts para extrair dados das LOAs para os anos de 2022 a 2025. Essas LOAs correspondem à legislatura de 2021-2024, eleita nas eleições de 2020.

Serão extraídas também a relação de vereadores que foram proponentes de cada emenda.

## extração de dados
Extraindo os arquivos da LOA

In [1]:
import pandas as pd
from typing import Optional
import plotly.graph_objects as go
import os

from utils.download_file import download_csv_as_dataframe

from config import DATA_FOLDER, GRAFICOS_FOLDER 


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
class DownloadLoa:

    base_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads'

    files = {

        2025 : 'LOA.25_VersionamentoDespesa.csv',
        2024 : 'OrcamentoAprovado_2024.csv',
        2023 : 'OrcamentoAprovado_2023.csv',
        2022 : 'OrcamentoAprovado_2022.csv'
    }

    def __build_url(self, year: int) -> str:
        """
        Builds the complete URL for downloading the LOA file for a given year.

        Args:
            year (int): The year for which the LOA file URL is to be built.

        Returns:
            str: The complete URL as a string.
        """
        endpoint = f'{year}/{self.files[year]}'
        return self.base_url + '/' + endpoint

    def __download(self, year: int, **read_kwargs)->pd.DataFrame:
        """
        Downloads the LOA file for a given year and returns it as a pandas DataFrame.

        Args:
            year (int): The year for which the LOA file is to be downloaded.

        Returns:
            pd.DataFrame: The downloaded LOA file as a pandas DataFrame.
        """



        url = self.__build_url(year)
        df = download_csv_as_dataframe(url, **read_kwargs)
        
        return df

    def __concat_year_dfs(self, year_list:[str], **read_kwargs)->pd.DataFrame:
        """
        Concatenates DataFrames for the specified years into a single DataFrame.

        Args:
            year_list (list): List of years for which DataFrames are to be concatenated.

        Returns:
            pd.DataFrame: The concatenated DataFrame.
        """
        dfs = [self.__download(year, **read_kwargs) for year in year_list]

        return pd.concat(dfs, ignore_index=True)

    def __describe_concated_df(self, df: pd.DataFrame) -> None:
        """
        Prints the shape, columns, and missing values for the given DataFrame.

        Args:
            df (pd.DataFrame): The DataFrame to describe.
        """
        print("Shape of DataFrame:", df.shape)
        print("Columns in DataFrame:", df.columns.tolist())
        print("Missing values per column:")
        print(df.isnull().sum())


    def __call__(self, year_list: Optional[[str]]=None, **read_kwargs)->pd.DataFrame:
        """
        Downloads and concatenates LOA files for the specified years.

        Args:
            year_list (list): List of years for which LOA files are to be downloaded.

        Returns:
            pd.DataFrame: The concatenated DataFrame containing the LOA data.
        """
        if year_list is None:
            year_list = self.files.keys()

        df = self.__concat_year_dfs(year_list, **read_kwargs)
        self.__describe_concated_df(df)

        return df


In [4]:
download_loa = DownloadLoa()

## limpeza de dados

padronizar colunas

In [5]:
df_loa = download_loa(encoding='latin-1', sep=';')

Shape of DataFrame: (18531, 40)
Columns in DataFrame: ['DOTAÇĂO', 'EXERCICIO_EMPRESA', 'EXERCICIO', 'ADMINISTRACAO', 'ORGAO', 'DESC_ORGAO', 'UNIDADE', 'DESC_UNIDADE', 'FUNCAO', 'DESC_FUNCAO', 'SUBFUNCAO', 'DESC_SUBFUNCAO', 'PROGRAMA', 'DESC_PROGRAMA', 'PAPA', 'PROJ_ATIV', 'DESC_PROJ_ATIV', 'CTA_DESPESA', 'DESC_CTA_DESPESA', 'FONTE', 'DESC_FONTE', 'TIPO_CREDITO', 'DESC_TIPO_CREDITO', 'COD_EX_FONT_REC', 'DESC_EX_FONT_REC', 'COD_DSTN_REC', 'TXT_DSTN_REC', 'COD_VINC_REC_PMSP', 'TXT_VINC_PMSP', 'COD_FONT_RDZD_REC', 'TXT_FONT_RDZD_REC', 'COD_NRO_EMND_DOTA', 'SALDO_ORÇ', 'AGENDA', 'DESC_AGENDA', 'EIXO', 'DESC_EIXO', 'PPA', 'IND_AGDA_ETGC', 'DOTAÇÃO']
Missing values per column:
DOTAÇĂO               4858
EXERCICIO_EMPRESA        0
EXERCICIO                0
ADMINISTRACAO            0
ORGAO                    0
DESC_ORGAO               0
UNIDADE                  0
DESC_UNIDADE             0
FUNCAO                   0
DESC_FUNCAO              0
SUBFUNCAO                0
DESC_SUBFUNCAO          

In [6]:
missing_values_count = df_loa.groupby("EXERCICIO").apply(lambda group: group.isnull().sum())


/tmp/ipykernel_295932/1849990876.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_values_count = df_loa.groupby("EXERCICIO").apply(lambda group: group.isnull().sum())


In [7]:
missing_values_percent = missing_values_count/df_loa.groupby("EXERCICIO").count()

In [8]:
missing_values_percent

,ADMINISTRACAO,AGENDA,COD_DSTN_REC,COD_EX_FONT_REC,COD_FONT_RDZD_REC,COD_NRO_EMND_DOTA,COD_VINC_REC_PMSP,CTA_DESPESA,DESC_AGENDA,DESC_CTA_DESPESA,DESC_EIXO,DESC_EX_FONT_REC,DESC_FONTE,DESC_FUNCAO,DESC_ORGAO,DESC_PROGRAMA,DESC_PROJ_ATIV,DESC_SUBFUNCAO,DESC_TIPO_CREDITO,DESC_UNIDADE,DOTAÇÃO,DOTAÇĂO,EIXO,EXERCICIO,EXERCICIO_EMPRESA,FONTE,FUNCAO,IND_AGDA_ETGC,ORGAO,PAPA,PPA,PROGRAMA,PROJ_ATIV,SALDO_ORÇ,SUBFUNCAO,TIPO_CREDITO,TXT_DSTN_REC,TXT_FONT_RDZD_REC,TXT_VINC_PMSP,UNIDADE
EXERCICIO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022,0.0,inf,inf,inf,inf,21.417062,inf,0.0,inf,0.0,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,inf,0.0,inf,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,inf,inf,inf,0.0
2023,0.0,inf,0.0,0.0,0.0,18.432000,0.0,0.0,inf,0.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,inf,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024,0.0,inf,0.0,0.0,0.0,15.142384,0.0,0.0,inf,0.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,inf,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025,0.0,inf,0.0,0.0,0.0,14.706564,0.0,0.0,inf,0.0,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,inf,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
cols = [col for col in missing_values_count.columns if missing_values_percent[col].sum() > 0 or missing_values_percent[col].isnull().all()]

In [10]:
missing_values_percent[cols]

,DOTAÇĂO,EXERCICIO,TIPO_CREDITO,DESC_TIPO_CREDITO,COD_EX_FONT_REC,DESC_EX_FONT_REC,COD_DSTN_REC,TXT_DSTN_REC,COD_VINC_REC_PMSP,TXT_VINC_PMSP,COD_FONT_RDZD_REC,TXT_FONT_RDZD_REC,COD_NRO_EMND_DOTA,AGENDA,DESC_AGENDA,EIXO,DESC_EIXO,DOTAÇÃO
EXERCICIO,,,,,,,,,,,,,,,,,,
2022,0.0,NaN,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,21.417062,inf,inf,inf,inf,inf
2023,inf,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.432000,inf,inf,inf,inf,0.0
2024,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.142384,inf,inf,inf,inf,inf
2025,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.706564,inf,inf,inf,inf,inf


In [11]:
cols_year = {year: set([col for col in df_loa if missing_values_percent.loc[year, col] < 100
            or pd.isnull(missing_values_percent.loc[year, col])
            ]) for year in range(2022,2026)}

In [12]:
for curr_year, cols_that_year in cols_year.items():
    
    cols_other_years=set()
    for year in cols_year:
        if year!= curr_year: cols_other_years.update(cols_year[year])

    print(f'Colunas especificas do ano: {year}', cols_that_year-cols_other_years)

Colunas especificas do ano: 2025 set()
Colunas especificas do ano: 2025 {'DOTAÇÃO'}
Colunas especificas do ano: 2025 set()
Colunas especificas do ano: 2025 set()


In [13]:
cols_comuns = []
for col in missing_values_percent.columns:
    for year, cols_that_year in cols_year.items():
        if col not in cols_that_year:
            print(f'Coluna {col} não está presente no ano {year}')
            break
    else:
        print(f'Coluna {col} está presente em todos os anos')
        cols_comuns.append(col)

Coluna ADMINISTRACAO está presente em todos os anos
Coluna AGENDA não está presente no ano 2022
Coluna COD_DSTN_REC não está presente no ano 2022
Coluna COD_EX_FONT_REC não está presente no ano 2022
Coluna COD_FONT_RDZD_REC não está presente no ano 2022
Coluna COD_NRO_EMND_DOTA está presente em todos os anos
Coluna COD_VINC_REC_PMSP não está presente no ano 2022
Coluna CTA_DESPESA está presente em todos os anos
Coluna DESC_AGENDA não está presente no ano 2022
Coluna DESC_CTA_DESPESA está presente em todos os anos
Coluna DESC_EIXO não está presente no ano 2022
Coluna DESC_EX_FONT_REC não está presente no ano 2022
Coluna DESC_FONTE está presente em todos os anos
Coluna DESC_FUNCAO está presente em todos os anos
Coluna DESC_ORGAO está presente em todos os anos
Coluna DESC_PROGRAMA está presente em todos os anos
Coluna DESC_PROJ_ATIV está presente em todos os anos
Coluna DESC_SUBFUNCAO está presente em todos os anos
Coluna DESC_TIPO_CREDITO não está presente no ano 2022
Coluna DESC_UNIDADE

Conclui-se que há uma alteração no schema dos dados a partir de 2024, sendo que 2023 tem algumas das colunas que passam a aparecer em 2024. 2022 tem apenas uma coluna, "DOTACAO", que não aparece nos demais. A partir de 2024, as alterações nas colunas são conhecidas, pois devem-se ao novo padrão para registro detalhado das fontes das dotações.

In [14]:
cols_comuns = set(cols_comuns)

In [15]:
cols_year[2025].difference(cols_comuns)

{'COD_DSTN_REC',
 'COD_EX_FONT_REC',
 'COD_FONT_RDZD_REC',
 'COD_VINC_REC_PMSP',
 'DESC_EX_FONT_REC',
 'DESC_TIPO_CREDITO',
 'DOTAÇĂO',
 'TIPO_CREDITO',
 'TXT_DSTN_REC',
 'TXT_FONT_RDZD_REC',
 'TXT_VINC_PMSP'}

In [16]:
cols_year[2024].difference(cols_comuns)

{'COD_DSTN_REC',
 'COD_EX_FONT_REC',
 'COD_FONT_RDZD_REC',
 'COD_VINC_REC_PMSP',
 'DESC_EX_FONT_REC',
 'DESC_TIPO_CREDITO',
 'DOTAÇĂO',
 'TIPO_CREDITO',
 'TXT_DSTN_REC',
 'TXT_FONT_RDZD_REC',
 'TXT_VINC_PMSP'}

In [17]:
cols_year[2023].difference(cols_comuns)

{'COD_DSTN_REC',
 'COD_EX_FONT_REC',
 'COD_FONT_RDZD_REC',
 'COD_VINC_REC_PMSP',
 'DESC_EX_FONT_REC',
 'DESC_TIPO_CREDITO',
 'DOTAÇÃO',
 'TIPO_CREDITO',
 'TXT_DSTN_REC',
 'TXT_FONT_RDZD_REC',
 'TXT_VINC_PMSP'}

In [18]:
cols_year[2022].difference(cols_comuns)

{'DOTAÇĂO'}

In [19]:
cols_comuns

{'ADMINISTRACAO',
 'COD_NRO_EMND_DOTA',
 'CTA_DESPESA',
 'DESC_CTA_DESPESA',
 'DESC_FONTE',
 'DESC_FUNCAO',
 'DESC_ORGAO',
 'DESC_PROGRAMA',
 'DESC_PROJ_ATIV',
 'DESC_SUBFUNCAO',
 'DESC_UNIDADE',
 'EXERCICIO',
 'EXERCICIO_EMPRESA',
 'FONTE',
 'FUNCAO',
 'IND_AGDA_ETGC',
 'ORGAO',
 'PAPA',
 'PPA',
 'PROGRAMA',
 'PROJ_ATIV',
 'SALDO_ORÇ',
 'SUBFUNCAO',
 'UNIDADE'}

Precisamos unificar e padronizar as colunas, mas vamos fazê-lo apenas para aquelas que fazem sentido para a análise, tentando manter principalmente as colunas que são comuns para todos os anos

In [20]:
colunas_padronizado = {
    'EXERCICIO' : 'exercicio',
    'COD_NRO_EMND_DOTA' : 'cod_emenda',
    'ADMINISTRACAO' : 'direta_ou_indireta',
    'ORGAO' : 'cod_orgao',
    'UNIDADE' : 'cod_unidade',
    'PROGRAMA' : 'cod_programa',
    'FUNCAO' : 'cod_funcao',
    'SUBFUNCAO' : 'cod_subfuncao',
    'PROJ_ATIV'  : 'cod_proj_atividade',
    'FONTE' : 'cod_fonte',
    'DESC_ORGAO' : 'desc_orgao',
    'DESC_UNIDADE' : 'desc_unidade',
    'DESC_PROGRAMA' : 'desc_programa',
    'DESC_FUNCAO' : 'desc_funcao',
    'DESC_SUBFUNCAO' : 'desc_subfuncao',
    'DESC_PROJ_ATIV' : 'desc_proj_atividade',
    'DESC_FONTE' : 'desc_fonte',
    'SALDO_ORÇ' : 'valor_orcado'
    }

In [21]:
cols_interesse = [col for col in colunas_padronizado.keys()]

df_loa = df_loa[cols_interesse].copy()
df_loa.rename(columns=colunas_padronizado, inplace=True)

In [22]:
df_loa.head()

,exercicio,cod_emenda,direta_ou_indireta,cod_orgao,cod_unidade,cod_programa,cod_funcao,cod_subfuncao,cod_proj_atividade,cod_fonte,desc_orgao,desc_unidade,desc_programa,desc_funcao,desc_subfuncao,desc_proj_atividade,desc_fonte,valor_orcado
0,2025,NaN,Administraçăo Direta,84,10,3003,10,304,2522,2,Fundo Municipal de Saúde,Fundo Municipal de Saúde,"Açőes e Serviços da Saúde em Atençăo Básica, E...",Saúde,Vigilância Sanitária,Manutençăo e Operaçăo de Vigilância em Saúde,Transferęncias Federais,76700000
1,2025,NaN,Administraçăo Direta,94,10,3005,18,541,7117,8,Fundo Especial do Meio Ambiente e Desenvolvime...,Fundo Especial do Meio Ambiente e Desenvolvime...,Promoçăo da Sustentabilidade Ambiental,Gestăo Ambiental,Preservaçăo e Conservaçăo Ambiental,Recuperaçăo de áreas degradadas e/ou contaminadas,Tesouro Municipal - Recursos Vinculados,310000
2,2025,NaN,Administraçăo Direta,94,10,3005,18,541,7117,8,Fundo Especial do Meio Ambiente e Desenvolvime...,Fundo Especial do Meio Ambiente e Desenvolvime...,Promoçăo da Sustentabilidade Ambiental,Gestăo Ambiental,Preservaçăo e Conservaçăo Ambiental,Recuperaçăo de áreas degradadas e/ou contaminadas,Tesouro Municipal - Recursos Vinculados,400000
3,2025,NaN,Administraçăo Direta,20,50,3009,26,453,1800,1,Secretaria Municipal de Mobilidade e Trânsito,Secretaria Executiva de Transporte e Mobilidad...,Melhoria da Mobilidade Urbana Universal,Transporte,Transportes Coletivos Urbanos,Eletrificaçăo da frota de veículos do Sistema ...,Operaçőes de Crédito,7071768
4,2025,NaN,Administraçăo Direta,20,50,3009,26,453,1800,1,Secretaria Municipal de Mobilidade e Trânsito,Secretaria Executiva de Transporte e Mobilidad...,Melhoria da Mobilidade Urbana Universal,Transporte,Transportes Coletivos Urbanos,Eletrificaçăo da frota de veículos do Sistema ...,Operaçőes de Crédito,19229040


### criando colunas de index

In [23]:
for col in ['exercicio', 'cod_orgao', 'cod_unidade', 'cod_proj_atividade']:
    print(col, df_loa[col].isnull().any())

exercicio False
cod_orgao False
cod_unidade False
cod_proj_atividade False


In [24]:
def fill_zeros(df:pd.DataFrame, col:str, num_zeros:int)->None:
    """
    Fills NaN values in the specified column of the DataFrame with zeros.

    Args:
        df (pd.DataFrame): The DataFrame to modify.
        col (str): The column name to fill with zeros.
    """
    df[col] = df[col].astype(int).astype(str)
    df[col] = df[col].str.zfill(num_zeros)

    print(col, df[col].str.len().unique())

In [25]:
fill_zeros(df_loa, 'exercicio', 4)
fill_zeros(df_loa, 'cod_orgao', 2)
fill_zeros(df_loa, 'cod_unidade', 2)
fill_zeros(df_loa, 'cod_proj_atividade', 4)

exercicio [4]
cod_orgao [2]
cod_unidade [2]
cod_proj_atividade [4]


In [26]:
df_loa['index'] = df_loa[['exercicio', 'cod_orgao', 'cod_unidade', 'cod_proj_atividade']].astype(str).agg('.'.join, axis=1)

In [27]:
df_loa['index'].sample(3)

1099     2025.84.10.2519
13335    2023.23.10.6028
5077     2024.84.10.9106
Name: index, dtype: object

O índice tem duplicadas, mas tudo bem. Ele não pode ter duplicadas na outra base, que identifica os parlamentares.

In [28]:
df_loa['index'].duplicated().any()

np.True_

In [29]:
df_loa['index'].duplicated().sum()

np.int64(10469)

### criando coluna de tratamento e avaliando

In [30]:
df_loa['is_emenda'] = df_loa['cod_emenda'].notnull()

In [31]:
df_loa['is_emenda'].mean()

np.float64(0.0551508283416977)

In [32]:
df_loa['is_emenda'].sum()

np.int64(1022)

In [33]:
emenda_count_per_year = df_loa.groupby('exercicio')['is_emenda'].sum()
emenda_count_per_year

exercicio
2022    211
2023    250
2024    302
2025    259
Name: is_emenda, dtype: int64

In [34]:
emenda_percent_per_year = df_loa.groupby('exercicio')['is_emenda'].mean()
emenda_percent_per_year

exercicio
2022    0.044609
2023    0.051462
2024    0.061949
2025    0.063668
Name: is_emenda, dtype: float64

In [ ]:

# Dados para o gráfico
anos = emenda_count_per_year.index.astype(int)
quantidade_emendas = emenda_count_per_year.values
percentual_emendas = emenda_percent_per_year.values * 100  # Converter para porcentagem

# Criar o gráfico
fig = go.Figure()

# Adicionar barras para a quantidade de emendas
fig.add_trace(go.Bar(
    x=anos,
    y=quantidade_emendas,
    name='Quantidade de Emendas',
    marker_color='blue'
))

# Adicionar anotações para o percentual em cada barra
i = 0
for ano, percentual in zip(anos, percentual_emendas):
    fig.add_annotation(
        x=ano,
        y=emenda_count_per_year[i],
        text=f'{percentual:.2f}%',
        showarrow=False,
        font=dict(size=12, color='black'),
        yshift=10  # Ajustar a posição vertical da anotação
    )
    i+=1

# Configurar layout
fig.update_layout(
    title='Evolução da Quantidade e Percentual de Emendas por Ano',
    xaxis = dict(
        title='Ano',
        tickvals=anos,  # Define os valores dos ticks no eixo x
        ticktext=[str(ano) for ano in anos]  # Define os rótulos dos ticks
    ),
    yaxis_title='Valores',
    barmode='group',
    legend_title='Métricas',
    template='plotly_white'
)

fig.write_image(os.path.join(GRAFICOS_FOLDER, 'qtd_emendas_ano.png'))

# Mostrar o gráfico
fig.show()

/tmp/ipykernel_295932/730940610.py:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



### padronizada final

In [ ]:
df_loa.dtypes

exercicio               object
cod_emenda             float64
direta_ou_indireta      object
cod_orgao               object
cod_unidade             object
cod_programa             int64
cod_funcao               int64
cod_subfuncao            int64
cod_proj_atividade      object
cod_fonte                int64
desc_orgao              object
desc_unidade            object
desc_programa           object
desc_funcao             object
desc_subfuncao          object
desc_proj_atividade     object
desc_fonte              object
valor_orcado             int64
index                   object
is_emenda                 bool
dtype: object

In [ ]:
for col in df_loa.columns:
    if col.startswith('cod'):
        df_loa[col] = df_loa[col].fillna('').astype(str)

In [ ]:
df_loa.dtypes

exercicio              object
cod_emenda             object
direta_ou_indireta     object
cod_orgao              object
cod_unidade            object
cod_programa           object
cod_funcao             object
cod_subfuncao          object
cod_proj_atividade     object
cod_fonte              object
desc_orgao             object
desc_unidade           object
desc_programa          object
desc_funcao            object
desc_subfuncao         object
desc_proj_atividade    object
desc_fonte             object
valor_orcado            int64
index                  object
is_emenda                bool
dtype: object

In [ ]:
df_loa.describe()

,valor_orcado
count,1.853100e+04
mean,2.245671e+07
std,2.090448e+08
min,0.000000e+00
25%,5.000000e+03
50%,2.000000e+05
75%,2.624757e+06
max,7.115266e+09


In [ ]:
df_loa.to_csv(os.path.join(DATA_FOLDER, 'df_loa.csv'), index=False, sep=';')

In [ ]:
!which python

/home/h-pgy/projects/artigo_regionalizacao_emendas/venv/bin/python
